In [2]:
! pip install plotly

  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB 217.9 kB/s eta 0:01:12
   ---------------------------------------- 0.0/15.7 MB 217.9 kB/s eta 0:01:12
   ---------------------------------------- 0.0/15.7 MB 151.3 kB/s eta 0:01:44
   ---------------------------------------- 0.0/15.7 MB 151.3 kB/s eta 0:01:44
   ---------------------------------------- 0.1/15.7 MB 181.6 kB/s eta 0:01:27
   ---------------------------------------- 0.1/15.7 MB 181.6 kB/s eta 0:01:27
   ---------------------------------------- 0.1/15.7 MB 183.3 kB/s eta 0:01:26
   -----------------------------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ------------------------- -------------- 9.9/15.7 MB 222.9 kB/s eta 0:00:27
   ------------------------- -------------- 9.9/15.7 MB 222.9 kB/s eta 0:00:27
   ------------------------- -------------- 9.9/15.7 MB 223.8 kB/s eta 0:00:26
   ------------------------- -------------- 9.9/15.7 MB 223.6 kB/s eta 0:00:26
   ------------------------- -------------- 9.9/15.7 MB 223.6 kB/s eta 0:00:26
   ------------------------- -------------- 9.9/15.7 MB 223.8 kB/s eta 0:00:26
   ------------------------- -------------- 9.9/15.7 MB 224.2 kB/s eta 0:00:26
   ------------------------- -------------- 9.9/15.7 MB 224.6 kB/s eta 0:00:26
   ------------------------- -------------- 10.0/15.7 MB 224.8 kB/s eta 0:00:26
   ------------------------- -------------- 10.0/15.7 MB 225.3 kB/s eta 0:00:26
   ------------------------- -------------- 10.0/15.7 MB 225.3 kB/s eta 0:00:26
   ------------------------- -------------- 10.0/15.7 MB 225.5 kB/s eta 0:00:26
   ------------------------- -------------- 10.0

In [15]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import os

In [285]:
df_natlang_eul = pd.DataFrame()
i = 1
for file in os.listdir('data/output/local/natural language sequence/euler'):
    df = pd.read_csv('data/output/local/natural language sequence/euler/'+file)
    df['sequence'] = i
    i += 1
    df_natlang_eul = pd.concat([df_natlang_eul,df])
df_natlang_eul_mean = pd.concat([df_natlang_eul[df_natlang_eul['sequence']==1].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']],
                                 df_natlang_eul[df_natlang_eul['sequence']==2].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']],
                                 df_natlang_eul[df_natlang_eul['sequence']==3].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']]],axis=1).T.set_index('sequence')

df_natlang_seq = pd.DataFrame()
i = 1
for file in os.listdir('data/output/local/natural language sequence/sequitur'):
    df = pd.read_csv('data/output/local/natural language sequence/sequitur/'+file)
    df['sequence'] = i
    i += 1
    df_natlang_seq = pd.concat([df_natlang_seq,df])
df_natlang_seq_mean = pd.concat([df_natlang_seq[df_natlang_seq['sequence']==1].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']],
                                 df_natlang_seq[df_natlang_seq['sequence']==2].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']],
                                 df_natlang_seq[df_natlang_seq['sequence']==3].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']]],axis=1).T.set_index('sequence')

fig = go.Figure(
    data=[
        go.Bar(
            name="De Bruijn Graph construction",
            x=["Sequence 1","Sequence 2","Sequence 3"],
            y=df_natlang_eul_mean['de_bruijn_graph_construction_time'],
            offsetgroup=0,
        ),
        go.Bar(
            name="Euler Path reconstruction",
            x=["Sequence 1","Sequence 2","Sequence 3"],
            y=df_natlang_eul_mean['euler_path_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_natlang_eul[df_natlang_eul['sequence']==i]['edit_distance']/df_natlang_eul[df_natlang_eul['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(1,4)), 
            textposition='outside',offsetgroup=0,
            base=df_natlang_eul_mean['de_bruijn_graph_construction_time'],
        ),

        go.Bar(
            name="Suffix Array construction",
            x=["Sequence 1","Sequence 2","Sequence 3"],
            y=df_natlang_seq_mean['suffix_array_construction_time'],
            offsetgroup=1,
        ),
        go.Bar(
            name="Adjacency Matrix construction",
            x=["Sequence 1","Sequence 2","Sequence 3"],
            y=df_natlang_seq_mean['adjacency_matrix_construction_time'],
            offsetgroup=1,
            base=df_natlang_seq_mean['suffix_array_construction_time'],
        ),go.Bar(
            name="Sequence reconstruction",
            x=["Sequence 1","Sequence 2","Sequence 3"],
            y=df_natlang_seq_mean['sequence_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_natlang_seq[df_natlang_seq['sequence']==i]['edit_distance']/df_natlang_seq[df_natlang_seq['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(1,4)), 
            textposition='outside',offsetgroup=1,
            base=df_natlang_seq_mean['adjacency_matrix_construction_time'],
        )
    ],
    layout=go.Layout(
        title="Natural Language Sequence Reconstruction: Suffix Array and Sequitur vs De Bruijn Graph and Euler Path",
        yaxis_title="time (s)"
    )
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'De Bruijn Graph construction',
              'offsetgroup': '0',
              'type': 'bar',
              'x': [Sequence 1, Sequence 2, Sequence 3],
              'y': array([0.00034584, 0.00018699, 0.0002245 ])},
             {'base': array([0.000345840183893824, 0.0001869913169315787, 0.00022450447082518533],
                            dtype=object),
              'name': 'Euler Path reconstruction',
              'offsetgroup': '0',
              'text': [Success Rate: 0.0%, Success Rate: 28.21%, Success Rate:
                       4.95%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [Sequence 1, Sequence 2, Sequence 3],
              'y': array([0.00013924, 0.00053954, 0.000242  ])},
             {'name': 'Suffix Array construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [Sequence 1, Sequence 2, Sequence 3],
              'y': array([2.36016512e-04, 9.81459618e-05, 1.15650654e-04])},
             {'base': array([0.00023601651191710248, 9.814596176146931e-05, 0.0001156506538391053],
                            dtype=object),
              'name': 'Adjacency Matrix construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [Sequence 1, Sequence 2, Sequence 3],
              'y': array([0.00111502, 0.00049771, 0.00067579])},
             {'base': array([0.0011150164604186545, 0.0004977076053619135, 0.0006757926940917647],
                            dtype=object),
              'name': 'Sequence reconstruction',
              'offsetgroup': '1',
              'text': [Success Rate: 100.0%, Success Rate: 100.0%, Success Rate:
                       100.0%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [Sequence 1, Sequence 2, Sequence 3],
              'y': array([0.01256799, 0.01170495, 0.01995712])}],
    'layout': {'template': '...',
               'title': {'text': ('Natural Language Sequence Reco' ... 'De Bruijn Graph and Euler Path')},
               'yaxis': {'title': {'text': 'time (s)'}}}
})

In [287]:
df_genlang_eul = pd.DataFrame()
i = 0
for file in os.listdir('data/output/local/generated sequence/euler'):
    df = pd.read_csv('data/output/local/generated sequence/euler/'+file)
    df['sequence'] = i
    i += 1
    df_genlang_eul = pd.concat([df_genlang_eul,df])
df_genlang_eul_mean = pd.DataFrame()
for i in range(5): df_genlang_eul_mean = pd.concat([df_genlang_eul_mean,df_genlang_eul[df_genlang_eul['sequence']==i].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']]],axis=1)
df_genlang_eul_mean = df_genlang_eul_mean.T.set_index('sequence')

df_genlang_seq = pd.DataFrame()
i = 0
for file in os.listdir('data/output/local/generated sequence/sequitur'):
    df = pd.read_csv('data/output/local/generated sequence/sequitur/'+file)
    df['sequence'] = i
    i += 1
    df_genlang_seq = pd.concat([df_genlang_seq,df])
df_genlang_seq_mean = pd.DataFrame()
for i in range(5): df_genlang_seq_mean = pd.concat([df_genlang_seq_mean,df_genlang_seq[df_genlang_seq['sequence']==i].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']]],axis=1)
df_genlang_seq_mean = df_genlang_seq_mean.T.set_index('sequence')

labels = list('Seed {}'.format(i) for i in range(5))

fig = go.Figure(
    data=[
        go.Bar(
            name="De Bruijn Graph construction",
            x=labels,
            y=df_genlang_eul_mean['de_bruijn_graph_construction_time'],
            offsetgroup=0,
        ),
        go.Bar(
            name="Euler Path reconstruction",
            x=labels,
            y=df_genlang_eul_mean['euler_path_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_genlang_eul[df_genlang_eul['sequence']==i]['edit_distance']/df_genlang_eul[df_genlang_eul['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(5)), 
            textposition='outside',offsetgroup=0,
            base=df_genlang_eul_mean['de_bruijn_graph_construction_time'],
        ),

        go.Bar(
            name="Suffix Array construction",
            x=labels,
            y=df_genlang_seq_mean['suffix_array_construction_time'],
            offsetgroup=1,
        ),
        go.Bar(
            name="Adjacency Matrix construction",
            x=labels,
            y=df_genlang_seq_mean['adjacency_matrix_construction_time'],
            offsetgroup=1,
            base=df_genlang_seq_mean['suffix_array_construction_time'],
        ),go.Bar(
            name="Sequence reconstruction",
            x=labels,
            y=df_genlang_seq_mean['sequence_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_genlang_seq[df_genlang_seq['sequence']==i]['edit_distance']/df_genlang_seq[df_genlang_seq['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(5)), 
            textposition='outside',offsetgroup=1,
            base=df_genlang_seq_mean['adjacency_matrix_construction_time'],
        )
    ],
    layout=go.Layout(
        title="Generated Pseudo-genomic Sequence Reconstruction: Suffix Array and Sequitur vs De Bruijn Graph and Euler Path",
        yaxis_title="time (s)"
    )
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'De Bruijn Graph construction',
              'offsetgroup': '0',
              'type': 'bar',
              'x': [Seed 0, Seed 1, Seed 2, Seed 3, Seed 4],
              'y': array([0.06601095, 0.06472188, 0.06708768, 0.07960882, 0.09069244])},
             {'base': array([0.06601094961166379, 0.06472188437238648, 0.06708767537147764,
                             0.07960881819116303, 0.09069244072001464], dtype=object),
              'name': 'Euler Path reconstruction',
              'offsetgroup': '0',
              'text': [Success Rate: 41.11%, Success Rate: 41.49%, Success Rate:
                       43.01%, Success Rate: 41.49%, Success Rate: 44.09%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [Seed 0, Seed 1, Seed 2, Seed 3, Seed 4],
              'y': array([0.17358735, 0.15085149, 0.15470059, 0.17692065, 0.20758155])},
             {'name': 'Suffix Array construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [Seed 0, Seed 1, Seed 2, Seed 3, Seed 4],
              'y': array([0.03108721, 0.03009903, 0.03255476, 0.0265025 , 0.02855194])},
             {'base': array([0.03108721256256099, 0.03009902715682979, 0.032554764747619584,
                             0.026502501964569047, 0.028551936149597123], dtype=object),
              'name': 'Adjacency Matrix construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [Seed 0, Seed 1, Seed 2, Seed 3, Seed 4],
              'y': array([0.4981147 , 0.49252175, 0.52816208, 0.40064641, 0.42023525])},
             {'base': array([0.498114697933197, 0.49252174615859984, 0.5281620788574218,
                             0.40064640998840334, 0.42023525238037107], dtype=object),
              'name': 'Sequence reconstruction',
              'offsetgroup': '1',
              'text': [Success Rate: 100.0%, Success Rate: 100.0%, Success Rate:
                       100.0%, Success Rate: 100.0%, Success Rate: 100.0%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [Seed 0, Seed 1, Seed 2, Seed 3, Seed 4],
              'y': array([0.10257199, 0.10541174, 0.11659167, 0.09064736, 0.0984034 ])}],
    'layout': {'template': '...',
               'title': {'text': ('Generated Pseudo-genomic Seque' ... 'De Bruijn Graph and Euler Path')},
               'yaxis': {'title': {'text': 'time (s)'}}}
})

In [288]:
df_genlang_eul = pd.DataFrame()
i = 0
for file in os.listdir('data/output/cluster/generated sequence/euler'):
    df = pd.read_csv('data/output/cluster/generated sequence/euler/'+file)
    df['sequence'] = i
    i += 1
    df_genlang_eul = pd.concat([df_genlang_eul,df])
df_genlang_eul_mean = pd.DataFrame()
for i in range(50): df_genlang_eul_mean = pd.concat([df_genlang_eul_mean,df_genlang_eul[df_genlang_eul['sequence']==i].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']]],axis=1)
df_genlang_eul_mean = df_genlang_eul_mean.T.set_index('sequence')

df_genlang_seq = pd.DataFrame()
i = 0
for file in os.listdir('data/output/cluster/generated sequence/sequitur'):
    df = pd.read_csv('data/output/cluster/generated sequence/sequitur/'+file)
    df['sequence'] = i
    i += 1
    df_genlang_seq = pd.concat([df_genlang_seq,df])
df_genlang_seq_mean = pd.DataFrame()
for i in range(50): df_genlang_seq_mean = pd.concat([df_genlang_seq_mean,df_genlang_seq[df_genlang_seq['sequence']==i].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']]],axis=1)
df_genlang_seq_mean = df_genlang_seq_mean.T.set_index('sequence')

labels = list('{}'.format(i) for i in range(50))

fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig.update_xaxes(title_text="seed",row=2, col=1)

fig.add_trace(go.Bar(
            name="De Bruijn Graph construction",
            x=labels,
            y=df_genlang_eul_mean['de_bruijn_graph_construction_time'],
            offsetgroup=0,
        ), row=1, col=1)
fig.add_trace(go.Bar(
            name="Euler Path reconstruction",
            x=labels,
            y=df_genlang_eul_mean['euler_path_reconstruction_time'],
            offsetgroup=0,
            base=df_genlang_eul_mean['de_bruijn_graph_construction_time'],
        ), row=1, col=1)
fig.add_trace(go.Bar(
            name="Suffix Array construction",
            x=labels,
            y=df_genlang_seq_mean['suffix_array_construction_time'],
            offsetgroup=1,
        ), row=1, col=1)
fig.add_trace(go.Bar(
            name="Adjacency Matrix construction",
            x=labels,
            y=df_genlang_seq_mean['adjacency_matrix_construction_time'],
            offsetgroup=1,
            base=df_genlang_seq_mean['suffix_array_construction_time'],
        ), row=1, col=1)
fig.add_trace(go.Bar(
            name="Sequence reconstruction",
            x=labels,
            y=df_genlang_seq_mean['sequence_reconstruction_time'],
            offsetgroup=1,
            base=df_genlang_seq_mean['adjacency_matrix_construction_time'],
        ), row=1, col=1)
fig.update_layout(title_text="Generated Pseudo-genomic Sequence Reconstruction: Suffix Array and Sequitur vs De Bruijn Graph and Euler Path")
fig.update_yaxes(title_text="time (s)",row=1, col=1)

fig.add_trace(go.Bar(
            name="De Bruijn Graph and Euler Path",
            x=labels,
            y=list(round((1-(df_genlang_eul[df_genlang_eul['sequence']==i]['edit_distance']/df_genlang_eul[df_genlang_eul['sequence']==i]['target_sequence_length'])).mean()*100,2) for i in range(50)),
            offsetgroup=0
        ),row=2,col=1)
fig.add_trace(go.Bar(
            name="Suffix Array and Sequitur",
            x=labels,
            y=list(round((1-(df_genlang_seq[df_genlang_seq['sequence']==i]['edit_distance']/df_genlang_seq[df_genlang_seq['sequence']==i]['target_sequence_length'])).mean()*100,2) for i in range(50)),
            offsetgroup=1
        ),row=2,col=1)
fig.update_yaxes(title_text="success rate (%)",row=2, col=1)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'De Bruijn Graph construction',
              'offsetgroup': '0',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'xaxis': 'x',
              'y': array([0.04946767, 0.05068754, 0.05094543, 0.05155022, 0.04933208, 0.04953132,
                          0.05082099, 0.0493269 , 0.0511779 , 0.04962186, 0.04897659, 0.04932527,
                          0.05060222, 0.04923525, 0.05091548, 0.04905209, 0.04914762, 0.05091383,
                          0.04890543, 0.05110241, 0.05014581, 0.05099751, 0.05040769, 0.05050881,
                          0.05053888, 0.05091355, 0.04872202, 0.05193091, 0.05049528, 0.04872784,
                          0.0503533 , 0.04894658, 0.0516667 , 0.04927005, 0.04750701, 0.04597135,
                          0.04597557, 0.04566875, 0.04689868, 0.0502646 , 0.0493411 , 0.04978327,
                          0.04964488, 0.05139933, 0.05195317, 0.04190841, 0.04078039, 0.04305919,
                          0.04140482, 0.04102579]),
              'yaxis': 'y'},
             {'base': array([0.0494676704883575, 0.05068754138946528, 0.05094542832374569,
                             0.05155022077560421, 0.04933207616806026, 0.04953132333755488,
                             0.05082099208831783, 0.04932690019607539, 0.05117789688110347,
                             0.04962185740470881, 0.048976585578918416, 0.04932526922225948,
                             0.050602221250534006, 0.04923524537086483, 0.0509154774665832,
                             0.04905208568572994, 0.04914761972427364, 0.0509138347625732,
                             0.04890543403625484, 0.05110241484642025, 0.0501458117485046,
                             0.05099751243591305, 0.050407691574096636, 0.05050881261825557,
                             0.050538882112503, 0.050913550043106034, 0.04872202353477474,
                             0.051930910158157303, 0.05049528145790096, 0.04872783775329585,
                             0.05035329623222347, 0.04894657616615291, 0.05166670222282406,
                             0.04927005066871639, 0.047507010602951, 0.04597134728431697,
                             0.04597556781768795, 0.04566874814033504, 0.04689867596626278,
                             0.05026459927558894, 0.049341097450256305, 0.049783271217346145,
                             0.04964487729072566, 0.051399329805374094, 0.05195316996574397,
                             0.041908410930633494, 0.04078038516044612, 0.04305919079780574,
                             0.041404818773269604, 0.04102579274177546], dtype=object),
              'name': 'Euler Path reconstruction',
              'offsetgroup': '0',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'xaxis': 'x',
              'y': array([0.37098191, 0.32864119, 0.33192337, 0.34286134, 0.32516664, 0.3352716 ,
                          0.32465526, 0.35320124, 0.33489816, 0.32547577, 0.35588249, 0.33456359,
                          0.36827451, 0.32648667, 0.33611551, 0.33917173, 0.32494954, 0.33892882,
                          0.34578616, 0.35202835, 0.32816992, 0.32923823, 0.32410799, 0.36174988,
                          0.33098909, 0.35362173, 0.36139868, 0.3346416 , 0.34091947, 0.33960591,
                          0.33696908, 0.3535865 , 0.35087208, 0.342612  , 0.32293378, 0.31112869,
                          0.29343243, 0.2955499 , 0.3229966 , 0.34808621, 0.32991147, 0.34249805,
                          0.34246361, 0.34983006, 0.32875213, 0.28943785, 0.28770513, 0.27

In [312]:
df_realseq_eul = pd.DataFrame()
i = 0
for file in os.listdir('data/output/cluster/real sequence/euler'):
    df = pd.read_csv('data/output/cluster/real sequence/euler/'+file)
    df['sequence'] = i
    i += 1
    df_realseq_eul = pd.concat([df_realseq_eul,df])
df_realseq_eul_mean = pd.DataFrame()
for i in range(6): df_realseq_eul_mean = pd.concat([df_realseq_eul_mean,df_realseq_eul[df_realseq_eul['sequence']==i].groupby("k").mean().mean()[["sequence","edit_distance","de_bruijn_graph_construction_time",'euler_path_reconstruction_time']]],axis=1)
df_realseq_eul_mean = df_realseq_eul_mean.T.set_index('sequence')

df_real_seq = pd.DataFrame()
i = 0
for file in os.listdir('data/output/cluster/real sequence/sequitur'):
    df = pd.read_csv('data/output/cluster/real sequence/sequitur/'+file)
    df['sequence'] = i
    i += 1
    df_real_seq = pd.concat([df_real_seq,df])
df_real_seq_mean = pd.DataFrame()
for i in range(6): df_real_seq_mean = pd.concat([df_real_seq_mean,df_real_seq[df_real_seq['sequence']==i].mean()[["sequence","edit_distance","suffix_array_construction_time",'adjacency_matrix_construction_time','sequence_reconstruction_time']]],axis=1)
df_real_seq_mean = df_real_seq_mean.T.set_index('sequence')

labels = list('{}'.format(i) for i in range(6))

fig = go.Figure(
    data=[
        go.Bar(
            name="De Bruijn Graph construction",
            x=labels,
            y=df_realseq_eul_mean['de_bruijn_graph_construction_time'],
            offsetgroup=0,
        ),
        go.Bar(
            name="Euler Path reconstruction",
            x=labels,
            y=df_realseq_eul_mean['euler_path_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_realseq_eul[df_realseq_eul['sequence']==i]['edit_distance']/df_realseq_eul[df_realseq_eul['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(6)),
            textposition='outside',offsetgroup=0,
            base=df_realseq_eul_mean['de_bruijn_graph_construction_time'],
        ),
        go.Bar(
            name="Suffix Array construction",
            x=labels,
            y=df_real_seq_mean['suffix_array_construction_time'],
            offsetgroup=1,
        ),
        go.Bar(
            name="Adjacency Matrix construction",
            x=labels,
            y=df_real_seq_mean['adjacency_matrix_construction_time'],
            offsetgroup=1,
            base=df_real_seq_mean['suffix_array_construction_time'],
        ),
        go.Bar(
            name="Sequence reconstruction",
            x=labels,
            y=df_real_seq_mean['sequence_reconstruction_time'],
            text=list("Success Rate: {}%".format(round((1-(df_real_seq[df_real_seq['sequence']==i]['edit_distance']/df_real_seq[df_real_seq['sequence']==i]['target_sequence_length'])).mean()*100,2)) for i in range(6)),
            textposition='outside',offsetgroup=1,
            base=df_real_seq_mean['adjacency_matrix_construction_time'],
        )
    ],
    layout=go.Layout(
        title="Raphanus sativus with Generated Reads: Suffix Array and Sequitur vs De Bruijn Graph and Euler Path",
        yaxis_title="time (s)"
    )
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'De Bruijn Graph construction',
              'offsetgroup': '0',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5],
              'y': array([2.61226663, 2.65840618, 2.64909406, 2.66195851, 2.66127944, 2.64254475])},
             {'base': array([2.612266627172144, 2.6584061766833806, 2.6490940644101397,
                             2.6619585080263093, 2.661279437716414, 2.642544750585789], dtype=object),
              'name': 'Euler Path reconstruction',
              'offsetgroup': '0',
              'text': [Success Rate: 0.01%, Success Rate: 0.01%, Success Rate:
                       0.01%, Success Rate: 0.01%, Success Rate: 0.01%, Success
                       Rate: 0.01%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5],
              'y': array([0.33540597, 0.34210571, 0.34665675, 0.34410124, 0.3425742 , 0.34427438])},
             {'name': 'Suffix Array construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5],
              'y': array([13.66321974, 13.5389039 , 13.7001111 , 13.72021866, 13.6174715 ,
                          13.90812485])},
             {'base': array([13.663219738006593, 13.538903903961181, 13.70011110305786,
                             13.720218658447266, 13.617471504211426, 13.908124852180482],
                            dtype=object),
              'name': 'Adjacency Matrix construction',
              'offsetgroup': '1',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5],
              'y': array([1062.36910439, 1073.00008652, 1066.01000216, 1058.08725832,
                          1048.33088028, 1078.49115772])},
             {'base': array([1062.369104385376, 1073.0000865221023, 1066.0100021600724,
                             1058.0872583150863, 1048.3308802843094, 1078.4911577224732],
                            dtype=object),
              'name': 'Sequence reconstruction',
              'offsetgroup': '1',
              'text': [Success Rate: 100.0%, Success Rate: 100.0%, Success Rate:
                       100.0%, Success Rate: 100.0%, Success Rate: 100.0%, Success
                       Rate: 100.0%],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0, 1, 2, 3, 4, 5],
              'y': array([216.3864481 , 216.98452675, 217.2779139 , 218.07359192, 217.90404272,
                          217.54738345])}],
    'layout': {'template': '...',
               'title': {'text': ('Raphanus sativus with Generate' ... 'De Bruijn Graph and Euler Path')},
               'yaxis': {'title': {'text': 'time (s)'}}}
})